# Assignment 2
## Group Members
1) Vipin Singh 19D070069 <br>
2) Sheel Shah 19D070052 <br>
3) Raghav Singhal 19D070049 <br>

## Input Netlist Format
### Text File
Primary Output<br>
Circuit Name<br>
Primary Input<br>
OUT_NODE GATE_TYPE IN_NODE<br>
....<br>
END<br>
Gate type can be:<br>
AND<br>
OR<br>
NOT<br>
XOR<br>

### Else 
You can edit the netlist give below in the format
["Primary Input", "Circuit Name","Primary Output","OUT_NODE GATE_TYPE IN_NODE", .... ,"END"]

In [1]:
def netlist_loader(path):
    f = open(path, "r")
    net = f.readlines()
    netlist = []
    for i in net:
        netlist.append(i.strip())
    return netlist
def input_file_generator(path):
    f = open(path, "r")
    input_list_preprocessing = f.readlines()
    input_list = []    
    for i in input_list_preprocessing:
        input_list.append(i.strip().split(" "))
    return input_list 



In [2]:
class circuit:
    
    def __init__(self,netlist):
        
        self.x = 0
        self.dominating_value = {"AND":0,"OR":1,"XOR":0,"NOT":'x'}
        self.non_dominating_value = {"AND":1,"OR":0,"XOR":0,"NOT":'x'}
        self.netlist = netlist
        #Primary outputs nodes
        self.PO = netlist[0].split(" ")
        self.CKT_NAME = netlist[1]
        
        #Primary Input nodes
        self.PI = netlist[2].split(" ")
        
        #Forming array of all gates in the ciruit
        self.GATES_ARRAY = self.make_gate_list()
        
        #Layer wise output is the ouput nodes available after each layer
        #Layer wise gate is the index of gate after each layer
        #Input Available will be used to map the value of each node to calculated
        self.layer_wise_output,self.layer_wise_gate,self.input_available =self.layer_circuit()
        
        self.output_node_generator()
        self.input_node_generator()
    
    def make_gate_list(self):
        #Makes the array of gate from the netlist
        GATES_ARRAY = []

        for i in range(3,len(self.netlist)):
            if self.netlist[i] =="END":
                break
            gate_specs = self.netlist[i].split(" ")
            gate_output = [gate_specs[0]]
            gate_type = gate_specs[1]
            gate_input = []
    
            for j in range(len(gate_specs)-2):
                gate_input.append(gate_specs[j+2])
            GATES_ARRAY.append([gate_output,gate_type,gate_input,0])
            
        #Output format is [[[output node],"Gate Type",[Input Node],0],[],...]
        #0 as the last elemet in this list to show it has not yet been placed on the flow graph
        return GATES_ARRAY
    
    def layer_circuit(self):
        # Helps to divide circuit in layer for execution
        #Layer wise output is the ouput nodes available after each layer
        layer_wise_output = []
        
        #Layer wise gate is the index of gate after each layer
        layer_wise_gate =[]
        
        #Input available keeps track of node available after layers
        input_available = []
        
        #Primary Input available before execution
        input_available += self.PI
        
        #Indicator to show all output nodes are available and the flow graph of circuit has been formed
        layer_not_completed = True

        while layer_not_completed:
            
            #ouput nodes available after this layer
            curr_layer = []
            
            #index of gate performed in this layer 
            curr_layer_gate = []
            
            #new input that next layer can use from this layer output
            input_available_next = []
            
            for i,gate in enumerate(self.GATES_ARRAY):
                
                #Check if gate already used if so go to next gate
                if gate[3] == 1:
                    
                    continue
                    
                gate_ready = True
                
                #Check if all input are available
                for j in gate[2]:
                    if j not in input_available:
                        #Input not availabe so move to next gate
                        gate_ready = False
                        break 
                        
                #If gate can be used, add its output to inpu available list and index and output node to their respective list
                if gate_ready:
                    input_available_next += gate[0]
                    curr_layer+=gate[0]
                    curr_layer_gate += [i]
                    gate[3] = 1
                    
            input_available += input_available_next             
            layer_wise_output.append(curr_layer)
            layer_wise_gate.append(curr_layer_gate)
            
            #Check if all primary outputs are available or not
            for i in self.PO:
                if i not in input_available:
                    break;
                elif i == self.PO[-1]:
                    layer_not_completed = False
        return layer_wise_output, layer_wise_gate,input_available
    
    
    def gate_ouput_calculation(self,g_type,g_input,value):
    
    #Gives output of the current gate
        x_present = False
        d_present = False
        d_bar_present = False
        if g_type == "AND":
            for i in g_input:
                if value[i] == 0:
                    return 0
                if value[i] == 'x':
                    x_present = True
                if value[i] == 'd':
                    d_present = True
                if value[i] == 'd_':
                    d_bar_present = True
            if x_present:
                return 'x'
            elif d_present:
                return 'd'
            elif d_bar_present:
                return 'd_'
            return 1
        
        if g_type == "OR":
            for i in g_input:
                if value[i] == 1:
                    return 1
                if value[i] == 'x':
                    x_present = True
                if value[i] == 'd':
                    d_present = True
                if value[i] == 'd_':
                    d_bar_present = True
            if x_present:
                return 'x'
            elif d_present:
                return 'd'
            elif d_bar_present:
                return 'd_'
            return 0
    
        if g_type == "NOT":
            for i in g_input:
                if value[i] == 1:
                    return 0
                elif value[i] == 'x':
                    return 'x'
                elif value[i] == 'd':
                    return 'd_'
                elif value[i] == 'd_':
                    return'd'
                else:
                    return 1
                
        if g_type == "XOR":
            sum_v = 0
            n_x = 0
            for i in g_input:
                if g_input != 'x':
                    sum_v += value[i]
                else:
                    n_x +=1
            
            if n_x%2 == 1:
                return 'x'
            else:
                return sum_v%2
                
                
    def input_applied(self,input_list):
        nodes_value = {}
        for i in self.input_available:
            nodes_value[i] = -1
        for i in input_list:
            nodes_value[i[0]] = i[1]
        
        
        for layer_gate in self.layer_wise_gate:
            for j in layer_gate:
                gate_type = self.GATES_ARRAY[j][1]
                gate_output_node = self.GATES_ARRAY[j][0][0]
                gate_value = self.gate_ouput_calculation(gate_type,self.GATES_ARRAY[j][2],nodes_value)
                nodes_value[gate_output_node] = gate_value
        return nodes_value
    
    def output(self,inp):
        input_list = []
        for i in inp:
            value = i.split(" ")
            if value[1] != 'x':
                input_list.append([value[0],int(value[1])])
                
            else:
                input_list.append([value[0],value[1]])
        self.node_value = self.input_applied(input_list)
        #for i in self.PO:
            #print(i," = ",self.node_value[i])
        return self.node_value
            
    def outputs(self,inps):
        for inp in inps:
            input_list = []
            
            for i,inpt in enumerate(inp):
                if inpt != 'x':
                    input_list.append([self.PI[i],int(inpt)])
                else:
                    input_list.append([self.PI[i],inpt])
                print(self.PI[i], "=", int(inpt),end = " ")
            self.node_value = self.input_applied(input_list)
            for i in self.PO:
                print(i," = ",self.node_value[i],end = " ")
            print("\n",end="")
            
    #Considering only normal gate
    def output_node_generator(self):
        self.output_mapping = []
        for node in self.GATES_ARRAY:
            self.output_mapping.append(node[0][0])
        #print(self.output_mapping)
            
    def input_node_generator(self):
        self.input_mapping = {}
        for i,nodes in enumerate(self.GATES_ARRAY):
            for node in nodes[2]:
                if node in self.input_mapping.keys():
                    self.input_mapping[node]+=[i]
                else:
                    self.input_mapping[node]=[i]
        #print(self.input_mapping)
    ###
    def input_init(self):
        inp = {}
        for pi in self.PI:
            inp[pi] = "x"
        return inp    
        
    def input_generator(self,input_values):
        input_str_list = []
        for i in self.PI:
            i_value = self.input_values[i]
            curr_str = i+" "
            if i_value == 'x':
                curr_str += 'x'
            else:
                curr_str += (str(i_value))
            input_str_list.append(curr_str)
        return input_str_list
        
        
    def node_validity_checker(self,obj_node,obj_value):
        
        if self.node_value[obj_node] == obj_value:
            return 1
        elif self.node_value[obj_node] == 'x':
            return 0
        else:
            return -1
        
    def d_frontier_formation(self,obj_node):
        d_frontier_list = []
        #print(self.node_value)
        for i in self.input_mapping[obj_node]:
            possible = True
            for input_node in self.GATES_ARRAY[i][2]:
                if input_node != obj_node:
                    if self.node_value[input_node] == self.dominating_value[self.GATES_ARRAY[i][1]]:
                        possible = False
            if possible:
                d_frontier_list+=self.GATES_ARRAY[i][0]
        #print(d_frontier_list)
        return d_frontier_list
                    
                        
        
        
        
    def test_vector_generator(self,node,node_value):
        self.decision_stack = []
        self.input_values = self.input_init()
        input_string = self.input_generator(self.input_values)
        temp = self.output(input_string)
        self.TV_found = False
        self.backtracking = False
        self.FL_detectable = True
        if node in self.PO:
            self.objective(node,node_value,False)
        self.objective(node,node_value,True)
        
    def objective(self,node,node_value,d_frontier,next_node = False):
        if self.node_value[node] != 'x':
            return
        self.objective_achieved= False
        self.backtrace(node,node,node_value,0,d_frontier,next_node)

        
    def backtrace(self,node,obj_node,obj_node_value,n_count,d_frontier,next_node):
        #print(node,obj_node,obj_node_value,"backtrace Started")
        if node not in self.PI:
            if self.node_value[node] != 'x':

                return
            curr_gate = self.output_mapping.index(node)

            for inp_node in self.GATES_ARRAY[curr_gate][2]:
                if self.node_value[inp_node] != 'x' :
                    continue
                self.backtrace(inp_node,obj_node,obj_node_value,n_count,d_frontier,next_node)
                if self.objective_achieved:
                    return


                if self.TV_found or self.backtracking:
                    return
        else:

            if self.node_value[node] == 'x':

                self.backtracking = False
                new_value = (obj_node_value + n_count%2)%2

                self.decision_stack.append([node,new_value])
                
                self.input_values[node] = new_value
                input_string = self.input_generator(self.input_values)
                temp = self.output(input_string)
                node_flag = self.node_validity_checker(obj_node,obj_node_value)
                #print(node,new_value, node_flag,"NODE FLAG", d_frontier)
                if node_flag == 1:
                    
                    if d_frontier:
                        d_frontier_list = self.d_frontier_formation(obj_node)
                        if d_frontier_list:
                            for d_fronier_gate_output_node in d_frontier_list:
                                self.forward_prop(d_fronier_gate_output_node,obj_node) 
                    else:
                        self.objective_achieved = True
                        return False
                if node_flag == 0:
                    if d_frontier:
                        d_frontier_list = self.d_frontier_formation(obj_node)
                        

                        if d_frontier_list:
                               
                            self.backtrace(obj_node,obj_node,obj_node_value,0,d_frontier,next_node)
                    else:
                        self.backtrace(obj_node,obj_node,obj_node_value,0,d_frontier,next_node)
                        
                if self.TV_found:
                    return
                
                self.backtracking = False
                self.decision_stack.pop()
                new_value = (1+new_value)%2
                self.input_values[node] = new_value
                self.decision_stack.append([node,new_value])
                input_string = self.input_generator(self.input_values)
                temp = self.output(input_string)
                
                node_flag = self.node_validity_checker(obj_node,obj_node_value)
                
                if node_flag == 1:
                    if d_frontier:
                        d_frontier_list = self.d_frontier_formation(obj_node)
                        if d_frontier_list:
                            for d_fronier_gate_output_node in d_frontier_list:
                                self.forward_prop(d_fronier_gate_output_node,obj_node)
                    else:
                        self.objective_achieved = True
                        return False
                    
                if node_flag == 0:
                    if d_frontier:
                        d_frontier_list = self.d_frontier_formation(obj_node)
                        if d_frontier_list:
                            self.backtrace(obj_node,obj_node,obj_node_value,0,d_frontier,next_node)
                    else:
                        self.backtrace(obj_node,obj_node,obj_node_value,0,d_frontier,next_node)
                        
                if self.TV_found:
                    return
                            
                self.decision_stack.pop()
                self.node_value[node] = 'x'
                self.input_string = self.input_generator(self.input_values)
                temp = self.output(input_string)
                
                if not self.decision_stack:
                    self.TV_found = True
                    self.FL_detectable = False
                
                self.backtracking = True
                
    def forward_prop(self,node,prev_node):
        self.objective_achieved = False
        curr_gate = self.output_mapping.index(node)

        for i in self.GATES_ARRAY[curr_gate][2]:
            
            if self.TV_found == True:
                return

            if i != prev_node:
                
                if self.node_value[i] == 'x':
                    temp1= self.objective(i,self.non_dominating_value[self.GATES_ARRAY[curr_gate][1]],False,next_node = node)
                    self.objective_achieved = False
                    if temp1 == False:
                        return False
                elif self.node_value[i] == self.dominating_value[self.GATES_ARRAY[curr_gate][1]]:
                    return False
                else:
                    
                    continue
        if node in self.PO:
            self.TV_found = True
        else:
            d_frontier_list = self.d_frontier_formation(node)
            if d_frontier_list:
                for d_fronier_gate_output_node in d_frontier_list:
                    self.forward_prop(d_fronier_gate_output_node,node) 

Give Fault Location and SA fault type in Fault_Location and SA variables given below

In [3]:
netlist = ["H","CKT_NAME","A B C D X","E AND A B","G AND C D","F OR E X","H OR G F","END"]
ckta = circuit(netlist)
Fault_Location  = "A"
SA = 1
ckta.test_vector_generator(Fault_Location,(SA+1)%2)

print("*********************")
if ckta.TV_found and ckta.FL_detectable:
    print(ckta.input_values)
print(ckta.TV_found and ckta.FL_detectable) 
print("*********************")

*********************
{'A': 0, 'B': 1, 'C': 0, 'D': 'x', 'X': 0}
True
*********************
